# Information Retrieval Project4

In [1]:
import os
import numpy as np

In [2]:
path = './train'
label = list(os.listdir(path))

In [3]:
label

['talk.politics.mideast',
 'rec.autos',
 'comp.sys.mac.hardware',
 'alt.atheism',
 'rec.sport.baseball',
 'comp.os.ms-windows.misc',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.med',
 'talk.politics.misc',
 'rec.motorcycles',
 'comp.windows.x',
 'comp.graphics',
 'comp.sys.ibm.pc.hardware',
 'sci.electronics',
 'talk.politics.guns',
 'sci.space',
 'soc.religion.christian',
 'misc.forsale',
 'talk.religion.misc']

In [4]:
len(label)

20

In [5]:
docu = {}

In [6]:
!pwd

/Users/sunqiushi/信息检索/Project4/lab4


In [7]:
# 读入训练集
def get_filelist(dir):
    Filelist = []
    for home, dirs, files in os.walk(dir):
        for filename in files:
            Filelist.append(os.path.join(home, filename))
    return Filelist

In [8]:
for labelname in label:
    files = get_filelist(path+'//'+labelname)
    temp = []
    for f in files:
        try:
            d = open(f, 'r', encoding="utf8")
            temp.append(d.read())
        except:
            continue
    docu[labelname] = temp
    d.close()

1. 分词
2. 统计词频

In [9]:
# 分词统计
dic = {}
dic_doc = {}
df = {}
for lb in label:
    dic_for_each = {}
    for p in docu[lb]:
        for w in p.split():
            w_low = w  # w.lower()
            if w_low in dic_for_each:
                dic_for_each[w_low] += 1
            else:
                dic_for_each[w_low] = 1

            if w in dic:
                dic[w_low] += 1
            else:
                dic[w_low] = 1
        for w in set(p.split()):
            if w in df:
                df[w] += 1
            else:
                df[w] = 1
    # 嵌套词典
    dic_doc[lb] = dic_for_each

In [10]:
p_lb = {}
temp_dict = list(dic)
dict = []

In [11]:
len(temp_dict)

324450

In [12]:
max_freq = 1000 # 800，900，1000....s
# 一个可以调的超参数

In [13]:
for w in temp_dict:
    if df[w] < max_freq:
        dict.append(w)

In [14]:
num_doc = {}

for lb in label:
    num_doc[lb] = 0

v = {}
for lb in label:
    v[lb] = set(dic_doc[lb])&set(dict)


for lb in label:
    for w in v[lb]:
        num_doc[lb] += dic_doc[lb][w]

In [15]:
num_doc

{'talk.politics.mideast': 191464,
 'rec.autos': 80347,
 'comp.sys.mac.hardware': 62676,
 'alt.atheism': 104488,
 'rec.sport.baseball': 89986,
 'comp.os.ms-windows.misc': 94431,
 'rec.sport.hockey': 133506,
 'sci.crypt': 124474,
 'sci.med': 105481,
 'talk.politics.misc': 149480,
 'rec.motorcycles': 75803,
 'comp.windows.x': 104955,
 'comp.graphics': 108093,
 'comp.sys.ibm.pc.hardware': 74731,
 'sci.electronics': 75584,
 'talk.politics.guns': 125506,
 'sci.space': 114462,
 'soc.religion.christian': 133781,
 'misc.forsale': 68681,
 'talk.religion.misc': 118364}

In [16]:
# count of per class
num_doc = {}
for lb in label:
    num_doc[lb] = 0
    for w in dic_doc[lb].items():
        num_doc[lb] += w[1]

sum_doc = 0
for lb in label:
    sum_doc += len(docu[lb])


# cal post perior prob
p_lb = {}
for lb in label:
    temp = {}
    for word in v[lb]:
        word_low = word #w.lower()
        temp[word_low] = (dic_doc[lb][word_low] + 0.9)/(num_doc[lb]+len(dict))
    p_lb[lb] = temp

In [17]:
# 读入测试集路径
test_path = './test-all-in-one'

In [18]:
# 读入测试集test-all-in-one
def get_test_filelist(dir):
    Filelist = []
    for home, dirs, files in os.walk(dir):
        for filename in files:
#             print(filename)
            Filelist.append(os.path.join(home, filename))
    return Filelist

In [19]:
all_test_files = get_test_filelist(test_path)

In [20]:
len(all_test_files)

3993

In [21]:
len('./test-all-in-one/')

18

In [22]:
len('cbd055b607cdb0cd0d38fe46a2d8a2fe')

32

In [23]:
# all_test_files

In [24]:
# 进行预测
file = open('./10185501402.txt','w',encoding='utf8')

for test_file in all_test_files:
    try:
        d = open(test_file, 'r', encoding="utf8")
        test = d.read()
        d.close()
    except:
        d.close()
    score_list = {}
    
    for lb in label:
        predict_score = np.log(len(docu[lb])/sum_doc)
#         print(predict_score)
        for w in test.split():
            if w in p_lb[lb]:
                predict_score += np.log(p_lb[lb][w])
            else:
                predict_score += np.log(1/(num_doc[lb]+len(dict)))
        score_list[lb] = predict_score
        
    score_list = sorted(score_list.items(),key=lambda x: x[1], reverse=True)
    
#     print(score_list[0][0])
    raw_prefix = f.split('\\')[-1]
#     print(raw_prefix)
    print(raw_prefix[18:50]+':'+score_list[0][0])
    print(raw_prefix[18:50]+':'+score_list[0][0], file=file)
    
file.close()

# 001427bb5c7de7f862e3e5719ec7eaa4

cbd055b607cdb0cd0d38fe46a2d8a2fe:talk.politics.mideast
38bb90cfef3662a464eac334754173a2:sci.med
62d491c8061e4f5daf87e69243416798:talk.politics.guns
8c7a62967cd1871feb7718b73adac71a:sci.crypt
5f02d5cc239f10e90c0e4b0fcb1e346f:talk.politics.guns
6fc4ef8b520cf5e5982926139fdae27d:talk.politics.misc
ba57244016cb711ac530424dbc014830:talk.politics.guns
6e0cf31ab8869301906f8b378a5b19b4:alt.atheism
d9197d33e4f4863a54f7628cab1099ed:talk.politics.misc
72c1ef79f08caeab93debe9f2922b49a:comp.sys.mac.hardware
65e29d694f90d581c1948b7afcd1121f:soc.religion.christian
68b55653cbaa18fa09d51c8a6aef7adf:rec.motorcycles
bed0a6e26f1de04c34a5b0dbad4a089e:sci.space
5532ac5525b31c759c92bbfc2dd209dc:comp.sys.mac.hardware
97703510653a43f9584c5a5edc900cfe:comp.graphics
a24a2e85ff7e300bd5ab486f17471b62:alt.atheism
c5e4e31661bbe7ab59146c76466cab36:comp.sys.ibm.pc.hardware
f32fae547b2951ba9cbd45ba24f8b6e6:comp.os.ms-windows.misc
396ea3a6830f7d4765ba0e1ec8aa3848:misc.forsale
8bd5a77b6a9f0f826e8e013dc8022a1d:sci.med
2f74